In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from itertools import cycle
import sklearn
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

In [4]:
#read tumor table
TumorSample_table = pd.read_csv("../EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena",sep='\t', index_col=0)
TumorSample_table = TumorSample_table.dropna()

In [5]:
Pathway_Alteration = pd.read_csv("../alterations/genomic_alteration_matrices.Pathway_level.tsv",sep='\t', index_col=0)
Pathway_Alteration = Pathway_Alteration.dropna()
Pathway_Alteration = Pathway_Alteration.astype(int)
Pathway_Alteration = Pathway_Alteration.reindex(columns=['Cell_Cycle', 'HIPPO', 'MYC', 'NOTCH', 'NRF2', 'PI3K', 'RTK RAS', 'TGF-Beta', 'TP53',  'WNT'])

In [6]:
#read curated pathway templates, descriptions of pathways
CELL_CYCLE_Table = pd.read_csv("../curated/curated_pathway_templates.Cell_Cycle.tsv",sep='\t', index_col=0)
HIPPO_Table = pd.read_csv("../curated/curated_pathway_templates.HIPPO.tsv",sep='\t', index_col=0)
MYC_Table = pd.read_csv("../curated/curated_pathway_templates.MYC.tsv",sep='\t', index_col=0)
NOTCH_Table = pd.read_csv("../curated/curated_pathway_templates.NOTCH.tsv",sep='\t', index_col=0)
NRF2_Table = pd.read_csv("../curated/curated_pathway_templates.NRF2.tsv",sep='\t', index_col=0)
PI3K_Table = pd.read_csv("../curated/curated_pathway_templates.PI3K.tsv",sep='\t', index_col=0)
RTK_RAS_Table = pd.read_csv("../curated/curated_pathway_templates.RTK_RAS.tsv",sep='\t', index_col=0)
TGF_BETA_Table = pd.read_csv("../curated/curated_pathway_templates.TGF-Beta.tsv",sep='\t', index_col=0)
TP53_Table = pd.read_csv("../curated/curated_pathway_templates.TP53.tsv",sep='\t', index_col=0)
WNT_Table = pd.read_csv("../curated/curated_pathway_templates.WNT.tsv",sep='\t', index_col=0)


CELL_CYCLE = tuple(set(CELL_CYCLE_Table.index).intersection(set(TumorSample_table.index))) #15 original curated genes, 0 lost due to NA
HIPPO  = tuple(set(HIPPO_Table.index).intersection(set(TumorSample_table.index)))          #38 original curated genes,  3 lost due to NA
MYC = tuple(set(MYC_Table.index).intersection(set(TumorSample_table.index)))               #13 original curated genes, 1 lost due to NA
NOTCH = tuple(set(NOTCH_Table.index).intersection(set(TumorSample_table.index)))           #71 original curated genes, 5 lost due to NA
NRF2 = tuple(set(NRF2_Table.index).intersection(set(TumorSample_table.index)))             #3 original curated genes, 0 lost due to NA
PI3K = tuple(set(PI3K_Table.index).intersection(set(TumorSample_table.index)))             #29 original curated genes, 1 lost due to NA
RTK_RAS = tuple(set(RTK_RAS_Table.index).intersection(set(TumorSample_table.index)))       #84 original curated genes, 5 lost due to NA
TGF_BETA = tuple(set(TGF_BETA_Table.index).intersection(set(TumorSample_table.index)))     #7 original curated genes, 0 lost due to NA
TP53 = tuple(set(TP53_Table.index).intersection(set(TumorSample_table.index)))             #6 original curated genes, 0 lost due to NA
WNT = tuple(set(WNT_Table.index).intersection(set(TumorSample_table.index)))               #68 original curated genes, 10 lost due to NA
ALL_PATHWAY_GENES = sum((CELL_CYCLE, HIPPO, MYC, NOTCH, NRF2, PI3K, RTK_RAS, TGF_BETA, TP53, WNT), ())
PATHWAYS = [CELL_CYCLE, HIPPO, MYC, NOTCH, NRF2, PI3K, RTK_RAS, TGF_BETA, TP53, WNT]

In [7]:
#Paradigm Gene Expression Scores
Paradigm_Pathway_Score = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/paradigm_pathway_activity.txt", sep="\t",index_col=0) 

#Somatic Mutations 
Somatic_Mutation_Table = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/somatic_mutation/mc3.v0.2.8.PUBLIC.nonsilentGene.xena",sep='\t', index_col=0)
Somatic_Mutation_Mapping = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/somatic_mutation/hugo_gencode_good_hg19_V24lift37_probemap",sep='\t', index_col=0)

#Copy Number Variance (CNV) tables ## -1 -> deletion, 0 -> normal, 1 -> overexpressed
CNV_Threshold = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/copy_number_variance/CNV_Threshold",sep='\t', index_col=0) ## what do 0,1,2 mean

#Methylation tables
Methylation_Table = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/methylation/HumanMethylation27",sep='\t', index_col=0)
ID_Gene_Map_Methylation = pd.read_csv("../tcga/TCGA_Tumor_Gene_Expression/methylation/illuminaMethyl27K_hg18_gpl8490_TCGAlegacy",sep='\t', index_col=0)

In [8]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])           #error term between actual and predicted, initally 0
                    for k in range(K):
                        #matrix obtained by rate_of_step(alpha) * gradient - regularization(beta term) of constant variables
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])  
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T
#possibly could improve with weighted alternating least square , regularized $http://ethen8181.github.io/machine-learning/recsys/1_ALSWR.html

In [9]:
patients = list(set(CNV_Threshold.columns).intersection(set(TumorSample_table.columns)).intersection(set(Somatic_Mutation_Table.columns)))
genes = list(set(ALL_PATHWAY_GENES).intersection(set(CNV_Threshold.index)).intersection(set(TumorSample_table.index)).intersection(set(Somatic_Mutation_Table.index))) 

TumorSample_table = TumorSample_table.loc[genes,patients] + (CNV_Threshold.loc[genes,patients].T.multiply(np.array(TumorSample_table.loc[genes,patients].std(axis=1)),axis=1)).T + (1 * (Somatic_Mutation_Table.loc[genes,patients].T.multiply(np.array(TumorSample_table.loc[genes,patients].std(axis=1)),axis=1)).T)

In [10]:
'''benchmark to compare CNV , mutation, versus expression'''

'''get mutation algorithm from alana to see if it is useful, driver '''

''' use expression to get a non binary score, and compare with paradigm'''

' use expression to get a non binary score, and compare with paradigm'

In [11]:
#input matrix so if tumor or onco is edited, normalize, consider different threshold for resultant matrix
#consider wals-r

#patients = set(Pathway_Alteration.index).intersection(set(TumorSample_table.columns))
###
patients = set(Pathway_Alteration.index).intersection(set(TumorSample_table.columns))

###
small_patients = random.sample(patients,10)
predictions = pd.DataFrame()

for pathway in PATHWAYS:
    R = np.array(TumorSample_table.loc[pathway,small_patients]) #gene (N) by patient (M)
    N = len(pathway) # number gene
    M = len(small_patients) # patients
    K = 1
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)
    nP, nQ = matrix_factorization(R, P, Q, K)
    nR = np.dot(nP, nQ.T)
    predictions = pd.concat([pd.DataFrame(nQ),predictions],axis=1)
predictions.columns = ['Cell_Cycle', 'HIPPO', 'MYC', 'NOTCH', 'NRF2', 'PI3K', 'RTK RAS', 'TGF-Beta','TP53',  'WNT']
predictions.index = small_patients

In [12]:
for pathway in predictions.columns:
    for patient in predictions.index:
        pathway_mean = predictions.loc[:, pathway].mean(axis=0)
        pathway_sd = predictions.loc[:, pathway].std(axis=0)
        if predictions.loc[patient, pathway] >  .3#pathway_mean + pathway_sd:
            predictions.loc[patient, pathway] = 1
        else: 
            predictions.loc[patient, pathway] = 0
predictions = predictions.astype(int)

SyntaxError: invalid syntax (<ipython-input-12-946c9f53b910>, line 5)

In [ ]:
#predictions.to_csv(sep="\t", path_or_buf='../temp.tsv')

In [ ]:
((predictions == Pathway_Alteration.loc[small_patients, predictions.columns]).sum().sum())/(len(small_patients)*10)

In [ ]:
from IPython.display import display, HTML


# for patient in set(Pathway_Alteration.index).intersection(set(Patient_Pathway_Score_Table.index)):
confusion_matrix = pd.DataFrame()    
for pathway in Pathway_Alteration.columns:
    a = pd.Series(Pathway_Alteration.loc[small_patients,pathway], name='Actual_' + pathway)
    b = pd.Series(predictions.loc[small_patients, pathway], name = 'Predicted_' + pathway)
    cm = pd.crosstab(a,b,rownames=['Predicted'])
    confusion_matrix = pd.concat([cm , confusion_matrix])

        
confusion_matrix = confusion_matrix.T.set_index(np.repeat('Actual', confusion_matrix.shape[1]), append=True).T
pd.concat([confusion_matrix] ,keys=("Cell_Cycle" ,"HIPPO", "MYC", "NOTCH", "NRF2", "PI3K", "RTK RAS", "TGF-Beta", "TP53", "WNT"))    

#balanced accuracy, prediction at about 50/50
accuracy = pd.DataFrame()
accuracy = pd.concat([confusion_matrix.T.apply(lambda x: x/sum(x)).T, accuracy], axis=1)

HTML('''
        <style>
            .accuracy tbody tr:nth-child(even) { background-color: lightblue; }
        </style>
        ''' + accuracy.to_html(classes="accuracy"))

In [ ]:
#Plot ROC for each pathway
n_classes = predictions.columns
patients = set(Pathway_Alteration.index).intersection(set(predictions.index))

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in n_classes:
    fpr[i], tpr[i], _ = roc_curve(np.array(Pathway_Alteration.loc[small_patients,i]) , np.array(predictions.loc[:,i]))
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(np.array(Pathway_Alteration.loc[small_patients,i]),  np.array(predictions.loc[:,i]).ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in n_classes]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in n_classes:
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= len(n_classes)

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
lw = 2

plt.figure(figsize=(10, 10))

plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'magenta', 'black', 'yellow', 'firebrick', 'red', 'pink'])
for i, color in zip(n_classes, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Pathway ROC')
plt.legend(loc="lower right")
plt.show()